# Part 2

(## Part 3 Report is as at below)

### Student Information
Name: 謝安亭

Student ID:110065516

GitHub ID:antinghsieh555

Kaggle name: AnterX

Kaggle private scoreboard snapshot:

[Snapshot](output/screenshot.png)

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dm2022-isa5810-lab2-homework/tweets_DM.json
/kaggle/input/dm2022-isa5810-lab2-homework/sampleSubmission.csv
/kaggle/input/dm2022-isa5810-lab2-homework/data_identification.csv
/kaggle/input/dm2022-isa5810-lab2-homework/emotion.csv


# Data Preprocess


In [4]:
# Read data
import numpy as np
import pandas as pd 
import json 
import os

path ='/kaggle/input/dm2022-isa5810-lab2-homework/'

data_id = pd.read_csv(path + "data_identification.csv") #1867535
emotion = pd.read_csv(path + "emotion.csv") #1455563
sampleSubmission = pd.read_csv(path + "sampleSubmission.csv") #411972

print('Start')
tweets = []
for line in open(path + 'tweets_DM.json', 'r'):
    line_ = json.loads(line)
    tweets.append(line_)


df = pd.DataFrame(columns=['tweet_id','text', 'hashtags'])
df["tweet_id"] = [tweet["_source"]["tweet"]["tweet_id"] for tweet in tweets]
df["text"] = [tweet["_source"]["tweet"]["text"] for tweet in tweets]
df["hashtags"] = [tweet["_source"]["tweet"]["hashtags"] for tweet in tweets]
print(df)
print('Finish!')

Start
         tweet_id                                               text  \
0        0x376b20  People who post "add me on #Snapchat" must be ...   
1        0x2d5350  @brianklaas As we see, Trump is dangerous to #...   
2        0x28b412  Confident of your obedience, I write to you, k...   
3        0x1cd5b0                Now ISSA is stalking Tasha 😂😂😂 <LH>   
4        0x2de201  "Trust is not the same as faith. A friend is s...   
...           ...                                                ...   
1867530  0x316b80  When you buy the last 2 tickets remaining for ...   
1867531  0x29d0cb  I swear all this hard work gone pay off one da...   
1867532  0x2a6a4f  @Parcel2Go no card left when I wasn't in so I ...   
1867533  0x24faed  Ah, corporate life, where you can date <LH> us...   
1867534  0x34be8c             Blessed to be living #Sundayvibes <LH>   

                                hashtags  
0                             [Snapchat]  
1          [freepress, TrumpLegacy, CNN]  


In [5]:
df.to_csv('/kaggle/working/tweets_DM.csv', sep='\t', encoding='utf-8')

In [6]:
# Split train data
train_df = pd.merge(data_id,emotion)
train_df = pd.merge(train_df,df)

if (train_df.shape[0] == data_id.loc[data_id['identification'] == 'train'].shape[0]):
    print('Correct Train Shape = ', train_df.shape[0])
else: print('Wrong Train Shape!')
train_df["emotion_code"] = train_df.emotion.map({"anger":0, "anticipation":1, "disgust":2, "fear":3, "sadness":4, "surprise":5, "trust":6, "joy":7})

# train_df

Correct Train Shape =  1455563


In [7]:
val_df = train_df.sample(frac=0.2)

In [8]:
# Split test data
test_df = data_id.loc[data_id['identification'] == 'test']
submit_temp = sampleSubmission.rename(columns={'id':'tweet_id'})
submit_temp = pd.merge(submit_temp, test_df)
test_df = pd.merge(submit_temp, df)

if (test_df.shape[0] == data_id.loc[data_id['identification'] == 'test'].shape[0]):
    print('Correct Test Shape = ', test_df.shape[0])
else: print('Wrong Test Shape!')
test_df


Correct Test Shape =  411972


,tweet_id,emotion,identification,text,hashtags
0,0x2c7743,surprise,test,When your friends offer to bring you food 😭💘 #...,"[loyal, real]"
1,0x2c1eed,surprise,test,I've never let any money problems stop me.. I ...,[]
2,0x2826ea,surprise,test,@KurtSchlichter Being a Hobby Historian Chelse...,[]
3,0x356d9a,surprise,test,#Cannabis offers a natural alternative treatme...,"[Cannabis, Marijuana, Weed, Hemp, Medicine, Le..."
4,0x20fd95,surprise,test,Last Friday off before school starts. I'm read...,[FridayFeeling]
...,...,...,...,...,...
411967,0x351857,surprise,test,Rahul Gandhi didn't even talk about Himachal P...,[]
411968,0x2c028e,surprise,test,"I'm ok with fixing a PC for someone, but not w...",[]
411969,0x1f2430,surprise,test,@brutofficiel I cannot believe this is happeni...,[]
411970,0x2be24e,surprise,test,I had Ecuadorian food and Thai food today <LH>,[]


In [9]:
train_df.to_csv('/kaggle/working/train_data.csv', index=False)
test_df.to_csv('/kaggle/working/test_data.csv', index=False)

# Read Train and Test Data


In [10]:
import numpy as np
import pandas as pd 
train = pd.read_csv('/kaggle/working/train_data.csv',
                    lineterminator='\n')
test = pd.read_csv('/kaggle/working/test_data.csv',
                   lineterminator='\n')
train['text'] = train['text'].apply(lambda x: x.lower())
test['text'] = test['text'].apply(lambda x: x.lower())

In [14]:
import re
emailsRegex = re.compile(r'[\w\.-]+@[\w\.-]+')
userMentionsRegex = re.compile(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)')
urlsRegex = re.compile('r(f|ht)(tp)(s?)(://)(.*)[.|/][^ ]+')
numsRegex = re.compile(r"\b\d+\b")

texts = train['text']

texts = texts.apply(lambda a:a.replace("<LH>",""))
texts = texts.apply(lambda a:userMentionsRegex.sub(' USER',a))
texts = texts.apply(lambda a:emailsRegex.sub(' EMAIL',a))
texts = texts.apply(lambda a:urlsRegex.sub(' URL',a))
texts = texts.apply(lambda a:numsRegex.sub(' NUM',a))

In [18]:
train['text'] = texts

,tweet_id,identification,emotion,text,hashtags,emotion_code
0,0x29e452,train,joy,huge respect🖒 USER talking about losing his d...,[],7
1,0x2b3819,train,joy,yoooo we hit all our monthly goals with the ne...,"['spateradio', 'app']",7
2,0x2a2acc,train,trust,USER USER_bch USER USER well done team 🌟 <...,[],6
3,0x2a8830,train,joy,come join USER on #pubg while he strives for ...,"['PUBG', 'GamersUnite', 'twitch', 'BeHealthy',...",7
4,0x20b21d,train,anticipation,USER blessings!my #strength little. my #bones...,"['strength', 'bones', 'God']",1
...,...,...,...,...,...,...
1455558,0x227e25,train,disgust,"USER such an inspirational talented person, w...",['rip'],2
1455559,0x293813,train,sadness,and still #libtards won't get off the guy's ba...,"['libtards', 'Hillary', 'lost', 'sad', 'growup...",4
1455560,0x1e1a7e,train,joy,when you sow #seeds of service or hospitality ...,"['seeds', 'Joy', 'GLTChurch']",7
1455561,0x2156a5,train,trust,USER will you be displaying some <lh> wares ...,[],6


In [19]:
train_df.to_csv('/kaggle/working/train_data.csv', index=False)

In [20]:
texts = test['text']

texts = texts.apply(lambda a:a.replace("<LH>",""))
texts = texts.apply(lambda a:userMentionsRegex.sub(' USER',a))
texts = texts.apply(lambda a:emailsRegex.sub(' EMAIL',a))
texts = texts.apply(lambda a:urlsRegex.sub(' URL',a))
texts = texts.apply(lambda a:numsRegex.sub(' NUM',a))

In [21]:
test['text'] = texts

In [22]:
test_df.to_csv('/kaggle/working/test_data.csv', index=False)

# Part 3

### 1. Input Data Preprocessing

First, I read the data from "tweets_DM.json".
Next, I merged "tweet_id", "text" and "hashtags" from "data_identification.csv" and "tweets_DM.json".

Then, concatenated the dataframe with the emotion label.

### 2.Text Preprocessing

After processing all the input data, the next thing is to process the text data.

1. Delete "LH" token in the text
2. Convert all text into lowercase
3. Replace @user token with USER token.
 e.g.@ERIC ===> USER
4. Replace url token with URL token.
5. Replace number of times token with NUM token.
e.g.25 times ===> NUM times

### 3. Train BERT model with training data
#### 3-1 : Load pre-train word embeddings model
In this method, I used pre-trained BERT-based word embedding vectors.

#### 3-2 : Input data preparation

1. Split the data into training set and testing set (test_size=0.2).
2. Use keras Tokenizer to tokenize text.
3. Use keras pad_sequences to pad the text (maxlen=50).
4. Utilize keras.utils.to_categorical for emotion label.(8 categoricals total)
5. Create word embedding matrix with pre-trained embeddings.(BERT embedding)

#### 3-3 : Build BERT model


### 4. Result